## Preguntas

* ¿Qué relaciones existen entre la edad de los peatones su localización/acción cuando ocurrió el accidente?

* Compare la relación entre género y razones de viaje.

* ¿Cómo influye el equipamiento de seguridad en la gravedad del accidente?

In [94]:
import json
import happybase
from pprint import pprint
from datetime import date 

In [104]:
def reconect():
    global connection
    try:
        connection.tables()
    except:
        connection = happybase.Connection('localhost')
        connection.open()
        print("reconected\n")
        
def create_table(name, dict_families):
    if name.encode('UTF-8') in connection.tables():
        connection.delete_table(name.encode('UTF-8'), True)
    connection.create_table(
        name,
        dict_families
    )
        
def drop_database():
    for table in connection.tables():
        connection.delete_table(table, True)
        print(table,"droped")
    print("database drop complete\n")

def add_family_data(table, row_key, column_family, column_names, values):
    column_data = {}
    row_key = row_key.encode('UTF-8')
    for i in range(len(column_names)):
        column = (column_family+':'+column_names[i]).encode('UTF-8')
        value = values[i].encode('UTF-8')
        column_data[column] = value

    table.put(row_key, column_data)

### Creacion de tablas iniciales

In [81]:
reconect()

drop_database()

create_table('actions_location', {'User': dict(), 'Situational': dict(), 'Date': dict()})
create_table('safety_consequences', {'Severity': dict(), 'Situational': dict(), 'User': dict(), 'Date': dict()})
create_table('enviromental_relevance', {'Enviroment': dict(), 'Place': dict(), 'Severity': dict()})

print(connection.tables())

b'actions_location' droped
b'enviromental_relevance' droped
b'safety_consequences' droped
database drop complete

[b'actions_location', b'enviromental_relevance', b'safety_consequences']


## Ultra Table

In [105]:
ultratable = {}
f = open("UltraTable.csv")
items = f.readline().replace("\"","").replace("\n","").replace("\ufeff","").split(",")
items_lenght = len(items)
for item in items:
    ultratable[item] = []
# print(users)

reconect()
actions_location_table = connection.table(b'actions_location')
safety_consequences_table = connection.table(b'safety_consequences')
enviromental_relevance_table = connection.table(b'enviromental_relevance')

for i in range(2):
    data = f.readline().replace("\"","").replace("\n","").split(",")
    dict_data = dict(zip(items, data))
#     pprint(dict_data)
#     print("")
    
    timestamp = '20'+dict_data['an'].zfill(2)+dict_data['mois'].zfill(2)+dict_data['jour'].zfill(2)+dict_data['hrmn']
  
    days_in_year = 365.2425
    accident_date = date(int('20'+dict_data['an'].zfill(2)), int(dict_data['mois']), int(dict_data['jour']))
    bdsplit = dict_data['year_on'].split("-")
    birth_date = date(int(bdsplit[2]), int(bdsplit[1]), int(bdsplit[0]))
    age = int((accident_date - birth_date).days / days_in_year)
    
    add_family_data(actions_location_table, 
                    str(i), 
                    'User',
                    ['sex', 'age'],
                    [dict_data['sexe'], str(age)])

f.close()

for key, data in actions_location_table.scan():
    print(key, data)

# users_dumps = json.dumps(users)
# f = open("users_cleaned.json","w")
# f.write(users_dumps)
# f.close()

reconected

b'0' {b'User:age': b'15', b'User:sex': b'Male'}
b'1' {b'User:age': b'33', b'User:sex': b'Female'}
